In [58]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('user_journey_raw.csv')

In [4]:
df.head()

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,2980248,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,2992252,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,3709807,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...


In [22]:
df['user_journey'][0].split('-')

['Homepage',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Log in',
 'Other']

In [6]:
df.shape

(9935, 4)

In [12]:
df.isnull().sum()

user_id              0
session_id           0
subscription_type    0
user_journey         0
dtype: int64

In [17]:
df['subscription_type'].unique()

array(['Annual', 'Monthly', 'Quarterly'], dtype=object)

In [19]:
df['user_journey'].nunique()

1841

# Data preprocessing

### 1. Remove duplicates from the user_journey column

In [111]:
def remove_page_duplicates(datafr, target_column):
    new_journey_col = []
    for journey in datafr[target_column]:
        list_ = journey.split('-')
        no_duplicates = []
        for page in list_:
            if page not in no_duplicates:
                no_duplicates.append(page)
        new_journey_col.append(no_duplicates)

    # turn the list of lists (new_journey_col) into a series to be able to join the words back with '-'
    s = pd.Series(new_journey_col)
    new_journey = s.str.join('-')

    # create a copy of the original dataframe with an updated column
    data = datafr.copy(deep=True)
    data[target_column] = new_journey
    return data

In [154]:
data = remove_page_duplicates(df, 'user_journey')

In [155]:
data.head()

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Other
1,1516,2980248,Annual,Other-Sign up-Log in
2,1516,2992252,Annual,Log in
3,1516,3070491,Annual,Homepage-Log in
4,1516,3709807,Annual,Log in


### 2. Group user_journey(s) by user_id

In [158]:
def group_by(datafr, group_by_col='user_id', target_col='user_journey', sessions='all', count_from='last'):
    data[target_col] = datafr.groupby([group_by_col])[target_col].transform(lambda x: "-".join(x.str.strip()))
    data_cl = datafr.copy(deep=True)
    data_cl.drop_duplicates(subset="user_id")
    return data_cl


In [150]:
# def inspect(x):
#     return print(type(x))


In [151]:
# data.groupby(['user_id'])[['user_journey']].transform(lambda x: inspect(x))

In [159]:
# data['user_journey'] = data.groupby(['user_id'])['user_journey'].transform(lambda x: "-".join(x.str.strip()))
# data = data.drop_duplicates(subset="user_id")
clean_data = group_by(data)
clean_data.head()

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Other-Other-Sign up-Log in-Log...
1,1516,2980248,Annual,Homepage-Log in-Other-Other-Sign up-Log in-Log...
2,1516,2992252,Annual,Homepage-Log in-Other-Other-Sign up-Log in-Log...
3,1516,3070491,Annual,Homepage-Log in-Other-Other-Sign up-Log in-Log...
4,1516,3709807,Annual,Homepage-Log in-Other-Other-Sign up-Log in-Log...


In [145]:
''' find out why groupby(['user_id'])[['user_journey']] was different -- the [[]] makes the result of the groupby
a dataframe, this then doesn't work on transform, as transform only works on one Series at a time. In our case, it
concatenates all the journeys grouoped by user. If we wanted to subtract 2 columns grouped by user, we'd have to use apply
'''
# find out why x.str.join('') was different -- str.join is a method for Series, join only works on iterables, not dataframes

" find out why groupby(['user_id'])[['user_journey']] was different -- the [[]] makes the result of the groupby\na dataframe, this then doesn't work on transform, as transform only works on one Series at a time. In our case, it\nconcatenates all the journeys grouoped by user. If we wanted to subtract 2 columns grouped by user, we'd have to use apply\n"

### 3. Remove all unnecessary pages (data scientist's choice)

In [160]:
def remove_pages(data, pages:list, target_col='user_journey'):
    kept_pages_col = []
    for list_ in data[target_col]:
        lst = list_.split('-')
        kept_pages = []
        for page in lst:
            if page not in pages:
                kept_pages.append(page)
        kept_pages_col.append(kept_pages)

    # turn the list of lists (new_journey_col) into a series to be able to join the words back with '-'
    s = pd.Series(kept_pages_col)
    new_pages = s.str.join('-')

    # create a copy of the original dataframe with an updated column
    stripped_pages_data = data.copy(deep=True)
    stripped_pages_data[target_col] = new_pages
    return stripped_pages_data

In [162]:
remove_pages(clean_data, ['Log in', 'Other'])

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Sign up-Homepage-Checkout-Checkout-Ch...
1,1516,2980248,Annual,Homepage-Sign up-Homepage-Checkout-Checkout-Ch...
2,1516,2992252,Annual,Homepage-Sign up-Homepage-Checkout-Checkout-Ch...
3,1516,3070491,Annual,Homepage-Sign up-Homepage-Checkout-Checkout-Ch...
4,1516,3709807,Annual,Homepage-Sign up-Homepage-Checkout-Checkout-Ch...
...,...,...,...,...
9930,509095,4487613,Annual,
9931,509095,4842565,Annual,
9932,509095,4843103,Annual,
9933,509095,4845316,Annual,
